# Maximum common substructure (MCS)

MolecularGraph.js implements essential MCS methods for cheminformatics applications.

- Node induced (MCIS) / Edge induced (MCES)
- Connected / disconnected
- Topological constraint (tdMCS)
- Graph-based local similarity (GLS)


In [1]:
import Pkg
Pkg.activate("..")
using MolecularGraph
using MolecularGraph.MolecularGraphModel


#  Download test data from PubChem

pubchemsdf("6047", "Levodopa")
pubchemsdf("74217", "3-Aminocoumarin")
pubchemsdf("6437877", "Cefditoren Pivoxil")
pubchemsdf("5481173", "Ceftazidime")


# Display a pair of mol images

function displayimgpair(img1, img2)
    """<div>
        <div style="float:left">$(img1)</div>
        <div style="float:left">$(img2)</div>
    </div>"""
end

file: Levodopa.mol already exists
file: 3-Aminocoumarin.mol already exists
file: Cefditoren Pivoxil.mol already exists
file: Ceftazidime.mol already exists


displayimgpair (generic function with 1 method)

## Basic examples

- Let's try MCS functionalities with very small molecules.
- Drop hydrogens from downloaded molecule data by `makehydrogensimplicit`

In [2]:
ldopa = sdftomol(joinpath(pubchem_dir, "Levodopa.mol"))
ldopa = graphmol(makehydrogensimplicit(ldopa))
svg1 = drawsvg(ldopa, 200, 200)

aminocoumarin = sdftomol(joinpath(pubchem_dir, "3-Aminocoumarin.mol"))
aminocoumarin = graphmol(makehydrogensimplicit(aminocoumarin))
svg2 = drawsvg(aminocoumarin, 200, 200)

display("text/html", displayimgpair(svg1, svg2))

OH 
 O 
 OH 
 NH 2 
 HO 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 O 
 NH 2 
 O

### Maximum common induced substructure (MCIS)

- `mcismol` method calculates maximum common induced substructure (MCIS).
- `mcismol` returns a tuple `(mapping, status)`.
- `mapping` (type `Dict{Int,Int}`) is a isomorphisim mapping of node(atom) indices between two molecules.
- `nodesubgraph` can be used to retrieve the matched subgraph and then display the substructure.
- `status`(type `Symbol`) shows the calculation status. `:done` means that the calculation was successfully finished.

In [3]:
(mapping, status) = mcismol(ldopa, aminocoumarin)

subg1 = nodesubgraph(ldopa, Set(keys(mapping)))
svg1 = drawsvg(subg1, 200, 200)

subg2 = nodesubgraph(aminocoumarin, Set(values(mapping)))
svg2 = drawsvg(subg2, 200, 200)

display("text/html", displayimgpair(svg1, svg2))
println("Status: $(status)")
println("MCIS size: $(length(mapping))")

O 
 OH 
 H 2 N 
 
 
 
 
 
 
 O 
 H 2 N 
 O

Status: done
MCIS size: 9


- Use `drawsvg` with `highlight` keyword to display the matched substructure.

In [4]:
svg1 = drawsvg(ldopa, 200, 200, highlight=subg1)
svg2 = drawsvg(aminocoumarin, 200, 200, highlight=subg2)

display("text/html", displayimgpair(svg1, svg2))
println("Status: $(status)")
println("MCIS size: $(length(mapping))")

OH 
 O 
 OH 
 NH 2 
 HO 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 O 
 NH 2 
 O

Status: done
MCIS size: 9


- Default node matcher function `nodematch` compares two atoms by their atomic symbol (`:C`, `:O`, `:N`, ...) and the number of $\pi$ electrons.
- The number of $\pi$ electrons gives as more intrinsic information of 3D geometry of the molecule than aparent bond order that cannot deal with conjugated system of the molecular orbital.
- Other atomic properties (charges, multiplicity, isotope, ...) of atoms are ignored by default.
- You can customize `nodematch` function and pass it to `mcismol` as an optional argument `nodematcher`.

### Maximum common edge-induced substructure (MCES)

- `mcesmol` calculates maximum common edge-induced substructure (MCES).
- Most chemists may feel that MCES is more intuitive than MCIS.
- `mcesmol` also returns a tuple `(mapping, status)`.
- Note that the result is an mapping of edge indices between two molecules. So use `edgesubgraph` to show the matched subgraph.

In [5]:
(mapping, status) = mcesmol(ldopa, aminocoumarin)

subg1 = edgesubgraph(ldopa, Set(keys(mapping)))
svg1 = drawsvg(ldopa, 200, 200, highlight=subg1)

subg2 = edgesubgraph(aminocoumarin, Set(values(mapping)))
svg2 = drawsvg(aminocoumarin, 200, 200, highlight=subg2)

display("text/html", displayimgpair(svg1, svg2))
println("Status: $(status)")
println("MCES size: $(length(mapping))")

OH 
 O 
 OH 
 NH 2 
 HO 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 O 
 NH 2 
 O

Status: done
MCES size: 8


### Connected MCS

- By default, `mcesmol` and `mcismol` calculate disconnected MCS.
- Pass keyword argument `connected=True` to them to enable connected MCS mode.
- Connected MCS is far faster than disconnected MCS but tend to be smaller in size.
- As disconnected MCS does not care the spatial relationship among matched fragments, connected MCS can be more appropriate option for some kind of applications.

In [6]:
(mapping, status) = mcismol(ldopa, aminocoumarin, connected=true)

subg1 = nodesubgraph(ldopa, Set(keys(mapping)))
svg1 = drawsvg(ldopa, 200, 200, highlight=subg1)

subg2 = nodesubgraph(aminocoumarin, Set(values(mapping)))
svg2 = drawsvg(aminocoumarin, 200, 200, highlight=subg2)

display("text/html", displayimgpair(svg1, svg2))
println("Status: $(status)")
println("Connected MCIS size: $(length(mapping))")

OH 
 O 
 OH 
 NH 2 
 HO 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 O 
 NH 2 
 O

Status: done
Connected MCIS size: 7


In [7]:
(mapping, status) = mcesmol(ldopa, aminocoumarin, connected=true)

subg1 = edgesubgraph(ldopa, Set(keys(mapping)))
svg1 = drawsvg(ldopa, 200, 200, highlight=subg1)

subg2 = edgesubgraph(aminocoumarin, Set(values(mapping)))
svg2 = drawsvg(aminocoumarin, 200, 200, highlight=subg2)

display("text/html", displayimgpair(svg1, svg2))
println("Status: $(status)")
println("Connected MCES size: $(length(mapping))")

OH 
 O 
 OH 
 NH 2 
 HO 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 O 
 NH 2 
 O

Status: done
Connected MCES size: 7


## Working with larger molecules

- Then, let's try it with a little larger molecules.

In [8]:
cefditoren = sdftomol(joinpath(pubchem_dir, "Cefditoren Pivoxil.mol"))
cefditoren = graphmol(makehydrogensimplicit(cefditoren))
svg1 = drawsvg(cefditoren, 250, 250)

ceftazidime = sdftomol(joinpath(pubchem_dir, "Ceftazidime.mol"))
ceftazidime = graphmol(makehydrogensimplicit(ceftazidime))
svg2 = drawsvg(ceftazidime, 250, 250)

display("text/html", displayimgpair(svg1, svg2))

S 
 NH 2 
 N 
 O 
 O 
 O 
 O 
 O 
 O 
 S 
 O 
 N 
 N 
 N 
 NH 
 S 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 S 
 NH 
 O 
 O 
 N 
 OH 
 O 
 O – 
 O 
 O 
 N 
 N 
 NH 2 
 N + 
 S

In [9]:
(mapping, status) = mcesmol(cefditoren, ceftazidime, timeout=10)

subg1 = edgesubgraph(cefditoren, Set(keys(mapping)))
svg1 = drawsvg(cefditoren, 250, 250, highlight=subg1)

subg2 = edgesubgraph(ceftazidime, Set(values(mapping)))
svg2 = drawsvg(ceftazidime, 250, 250, highlight=subg2)

display("text/html", displayimgpair(svg1, svg2))
println("Status: $(status)")
println("MCES size: $(length(mapping))")

S 
 NH 2 
 N 
 O 
 O 
 O 
 O 
 O 
 O 
 S 
 O 
 N 
 N 
 N 
 NH 
 S 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 S 
 NH 
 O 
 O 
 N 
 OH 
 O 
 O – 
 O 
 O 
 N 
 N 
 NH 2 
 N + 
 S

Status: timedout
MCES size: 21


- As disconnected MCS takes very long time,  keyword argument`timeout` was set to 10 second to abort MCS calculation.
- If it timed out, the status will be `timedout` and the result will be suboptimal common substructure size calculated so far.

In [10]:
using Profile
@profile mcesmol(cefditoren, ceftazidime, timeout=10)
Profile.print(mincount=1000)

7004 ./task.jl:259; (::getfield(IJulia, Symbol("##15#1...
 7004 ...fjEtl/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  7004 ./essentials.jl:741; invokelatest
   7004 ./essentials.jl:742; #invokelatest#1
    7004 ...c/execute_request.jl:67; execute_request(::ZMQ.Socket, ::...
     7003 .../SoftGlobalScope.jl:218; softscope_include_string(::Modu...
      7003 ./boot.jl:328; eval
       7001 ./none:0; (::getfield(MolecularGraph, Sym...
        7001 ...Graph.jl/src/mcs.jl:48; #mcesmol#40(::Base.Iterators.P...
         7001 ./none:0; (::getfield(MolecularGraph.Mol...
          6828 ...phism/cliquemcs.jl:87; #findmces#156(::Function, ::...
           6828 ./none:0; (::getfield(MolecularGraph.M...
            6827 ...c/graph/clique.jl:161; #maximalcliques#78
             6807 .../graph/clique.jl:104; expand!(::MolecularGraph.M...
              6806 .../graph/clique.jl:104; expand!(::MolecularGraph....
               6805 ...graph/clique.jl:104; expand!(::MolecularGraph....
                680

- The most costful call was `maximalcliques` which yields maximal cliques. Maximum clique detection problem is known to be NP-hard.

In [11]:
(mapping, status) = mcesmol(cefditoren, ceftazidime, connected=true)

subg1 = edgesubgraph(cefditoren, Set(keys(mapping)))
svg1 = drawsvg(cefditoren, 250, 250, highlight=subg1)

subg2 = edgesubgraph(ceftazidime, Set(values(mapping)))
svg2 = drawsvg(ceftazidime, 250, 250, highlight=subg2)

display("text/html", displayimgpair(svg1, svg2))
println("Status: $(status)")
println("Connected MCES size: $(length(mapping))")

S 
 NH 2 
 N 
 O 
 O 
 O 
 O 
 O 
 O 
 S 
 O 
 N 
 N 
 N 
 NH 
 S 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 S 
 NH 
 O 
 O 
 N 
 OH 
 O 
 O – 
 O 
 O 
 N 
 N 
 NH 2 
 N + 
 S

Status: done
Connected MCES size: 27


- Connected MCS is far faster than disconnected MCS.

In [13]:
(mapping, status) = mcesmol(cefditoren, ceftazidime, targetsize=15)

subg1 = edgesubgraph(cefditoren, Set(keys(mapping)))
svg1 = drawsvg(cefditoren, 250, 250, highlight=subg1)

subg2 = edgesubgraph(ceftazidime, Set(values(mapping)))
svg2 = drawsvg(ceftazidime, 250, 250, highlight=subg2)

display("text/html", displayimgpair(svg1, svg2))
println("Status: $(status)")
println("Connected MCES size: $(length(mapping))")

S 
 NH 2 
 N 
 O 
 O 
 O 
 O 
 O 
 O 
 S 
 O 
 N 
 N 
 N 
 NH 
 S 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 S 
 NH 
 O 
 O 
 N 
 OH 
 O 
 O – 
 O 
 O 
 N 
 N 
 NH 2 
 N + 
 S

Status: targetreached
Connected MCES size: 15


- Or you can use `targetsize` keyword argument to know that these compounds have at least the given size of the common substructure. This feature is quite useful in library screening.

## Topological constraint

- Disconnected MCS methods can detect as many matched fragments as possible, but it does not reflect spatial relationship of each fragments.
- On the other hand, connected MCS is too strict not to allow distant matches.
- Graph distance function can be used as a node attribute matcher to constrain the spatial arrangement of matched substructure fragments (topological constraint). This feature seems to be preferable in pharmacophore matching and structural similarity screening.
- Topological constraint also effectively decreases the calculation cost.
- Topological constraint can be used by passing keyword argument `topological=True`.
- Distance mismatch tolerance parameter $\theta$ is also available as the keyword argument `tolerance`.

In [14]:
(mapping, status) = mcesmol(cefditoren, ceftazidime, topological=true)

subg1 = edgesubgraph(cefditoren, Set(keys(mapping)))
svg1 = drawsvg(cefditoren, 250, 250, highlight=subg1)

subg2 = edgesubgraph(ceftazidime, Set(values(mapping)))
svg2 = drawsvg(ceftazidime, 250, 250, highlight=subg2)

display("text/html", displayimgpair(svg1, svg2))
println("Status: $(status)")
println("tcMCES size: $(length(mapping))")

S 
 NH 2 
 N 
 O 
 O 
 O 
 O 
 O 
 O 
 S 
 O 
 N 
 N 
 N 
 NH 
 S 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 S 
 NH 
 O 
 O 
 N 
 OH 
 O 
 O – 
 O 
 O 
 N 
 N 
 NH 2 
 N + 
 S

Status: done
tcMCES size: 28
